In [1]:
import numpy as np
from LoopStructural import GeologicalModel
import pandas as pd
from LoopStructural.visualisation import Loop3DView
# import loopstructuralmeshing
from loopstructuralmeshing import meshbuilder

In [2]:
data = pd.DataFrame(
    [
        [100, 100, 150, 0.17, 0, 0.98, 0, "strati"],
        [100, 100, 170, 0, 0, 0.86, 0, "strati3"],
        [100, 100, 100, 0, 0, 1, 0, "strati2"],
        [100, 100, 50, 0, 0, 1, 0, "nconf"],
        [100, 100, 50, 0, 0, 1, 0, "strati4"],
        [700, 100, 190, 1, 0, 0, np.nan, "fault"],
    ],
    columns=["X", "Y", "Z", "nx", "ny", "nz", "val", "feature_name"],
)

In [3]:
model = GeologicalModel(np.zeros(3), np.array([1000, 1000, 200]))
model.data = data
model.create_and_add_fault(
    "fault", 50, minor_axis=700, major_axis=3000, intermediate_axis=300, fault_center = [700, 500, 0],
)

model.create_and_add_foliation("strati", buffer=0.0)


                          projected onto fault surface estimating from fault normal


-----------------------------------------------------
strati 1 
-----------------------------------------------------
	0 regions
	0 faults.
	Fault enabled True

In [4]:
# tetmesh = loopstructuralmeshing.MeshFactory.create_mesh(model.bounding_box,n_elements=1e4)
tetmesh = meshbuilder.MeshFactory.create_mesh(model.bounding_box,n_elements=1e4)

In [5]:
# tetmesh.nodes/=1000

In [6]:
class TetMesh:
    def __init__(self, nodes, elements):
        self.nodes = nodes
        self.elements = elements
        self.point_data = {}
        self.element_data = {}
        self.material = np.zeros(self.elements.shape[0])
    def barycentre(self):
        return np.mean(self.nodes[self.elements],axis=1)
    def vtk(self):
        return create_pyvista_mesh(self.nodes,self.elements)
    def optimize(self):
        import meshio
        import pygmsh
        mesh = meshio.Mesh(points=self.nodes, cells=[("tetra", self.elements)])
        opt_mesh = pygmsh.optimize(mesh)
        self.nodes = opt_mesh.points
        self.elements = opt_mesh.cells[0].data
    def save(self, filename):
        import meshio
        meshio.write_points_cells(
            filename,
            self.nodes,
            {"tetra": self.elements},
            point_data=self.point_data,
            # cell_data=self.element_data,
        )

In [7]:
newmesh = TetMesh(tetmesh.nodes/100,tetmesh.elements)

In [8]:
from pyvista import CellType

def create_pyvista_mesh(nodes,elements):
    celltypes = np.full(elements.shape[0], fill_value=CellType.TETRA, dtype=np.uint8)
    pvtetmesh = pv.UnstructuredGrid(np.hstack([np.ones((elements.shape[0],1))+3,elements]).astype(int),celltypes,nodes)

    return pvtetmesh

In [9]:
def dotmeshtotetmesh(filename):

    import meshio
    optimesh = meshio.read(filename)
    return TetMesh(optimesh.points,optimesh.cells[0].data)
def savesol(filename,mesh,feature):
    vals = feature.evaluate_value(feature.model.scale(mesh.nodes)*100)
    with open(filename,'w') as file:

        file.write(f"""MeshVersionFormatted 1

    Dimension 3

    SolAtVertices
    {len(mesh.nodes)}
    1 1

    """)
        np.savetxt(file,vals)
        file.write(f"""

    End
            """)    

def writedotmesh(filename,mesh):
    with open(filename,'w') as file:
        file.write(f"""MeshVersionFormatted 2

MeshVersionFormatted 2
Dimension 3

Vertices
{len(mesh.nodes)}
""")

        np.savetxt(file,mesh.nodes)
        file.write(f"""/n
Tetrahedra
{len(mesh.elements)}
""")
        np.savetxt(file,np.hstack([mesh.elements.astype(int),mesh.material.astype(int)]))
        file.write(f"""/n""")

In [10]:
def writemultimaterials2(filename):
    with open(filename,'w') as file:
        file.write(f"""LSReferences
3

4 nosplit
5 1 2
6 3 7
""")
def writemultimaterials1(filename):
    with open(filename,'w') as file:
        file.write(f"""LSReferences
2

2 4 5
3 6 7 
""")  

In [11]:
newmesh.nodes.max()

10.0

In [12]:
import subprocess
import shutil
meshname = 'tetmesh1e2.mesh'
solname = 'tetmesh1e2_solution.sol'

savesol(solname,newmesh,model['fault'])
# add fault to mesh
isovalue = -0.01
newmesh.save(meshname)
subprocess.run(['mmg3d_O3', meshname,'-ls',str(isovalue),  '-sol', solname, '-out', 'meshout.mesh'])
mesh = dotmeshtotetmesh('meshout.mesh')
savesol(solname,mesh,model['fault'])
isovalue = .01
meshname='meshout.mesh'
writemultimaterials1('meshout.mmg3d')
subprocess.run(['mmg3d_O3',  meshname,'-ls',str(isovalue), '-sol', solname, '-out', 'meshout.mesh','-hmax','.7'])

# newmesh.save(meshname)
# mesh = dotmeshtotetmesh('meshout.mesh')
# import meshio
# meshio.read('meshout.mesh').write('meshout.vtk')
# mesh.save('optimized_mesh.vtk')



  -- MMG3D, Release 5.7.0 (Dec. 13, 2022) 
     Copyright (c) Bdx INP/CNRS/Inria/UPMC, 2004-
     Jun 26 2024 16:20:47

  -- INPUT DATA
  %% tetmesh1e2.mesh OPENED
  %% tetmesh1e2_solution.sol OPENED
  -- DATA READING COMPLETED.     0.071s

  -- MMG3DLS: INPUT DATA
  -- INPUT DATA COMPLETED.     0.000s

  -- PHASE 1 : ISOSURFACE DISCRETIZATION

  -- MESH QUALITY  22350
     BEST   0.996462  AVRG.   0.690485  WRST.   0.122418 (13922)
  -- PHASE 1 COMPLETED.     0.062s

  -- PHASE 2 : ANALYSIS
  -- PHASE 2 COMPLETED.     0.038s

  -- PHASE 3 : MESH IMPROVEMENT
  *** Manifold implicit surface.
                            2974 splitted,       83 collapsed,    12221 swapped, 5 iter.

  -- GRADATION : 1.300000 (2.300000)
                             368 splitted,     9146 collapsed,      806 swapped, 6 iter.
  *** Manifold implicit surface.
  -- PHASE 3 COMPLETED.     0.676s

  -- MESH QUALITY  79
     BEST   0.901454  AVRG.   0.494348  WRST.   0.188392 (40)

  -- MESH PACKED UP
     NUMBER 

Warning: Meshio doesn't know keyword RequiredVertices. Skipping.

Warning: Meshio doesn't know keyword Tangents. Skipping.

Warning: Meshio doesn't know keyword TangentAtVertices. Skipping.

Warning: Meshio doesn't know keyword Ridges. Skipping.

  -- MMG3D, Release 5.7.0 (Dec. 13, 2022) 
     Copyright (c) Bdx INP/CNRS/Inria/UPMC, 2004-
     Jun 26 2024 16:20:47

  -- INPUT DATA
  %% meshout.mesh OPENED
  %% tetmesh1e2_solution.sol OPENED

  %% meshout.mmg3d OPENED
  -- DATA READING COMPLETED.     0.028s

  -- MMG3DLS: INPUT DATA
  -- INPUT DATA COMPLETED.     0.000s

  -- PHASE 1 : ISOSURFACE DISCRETIZATION

  -- MESH QUALITY  79
     BEST   0.901454  AVRG.   0.494348  WRST.   0.188392 (79)
  -- PHASE 1 COMPLETED.     0.028s

  -- PHASE 2 : ANALYSIS
  -- PHASE 2 COMPLETED.     0.022s

  -- PHASE 3 : MESH IMPROVEMENT
  *** Manifold implicit surface.
                            1082 splitted,        0 collapsed,     2523 swapped, 6 iter.

  -- GRADATION : 1.300000 (2.300000)
                             172 splitted,      271 collapsed,      719 swapped, 6 iter.
         2072 splitted,     1232 collapsed,     4942 swapped,     9514 moved, 6 iter. 
  *** Manifold implicit surface.
  -- PHASE 3 COMPLETED.     0.401s

  -- MESH QU

CompletedProcess(args=['mmg3d_O3', 'meshout.mesh', '-ls', '0.01', '-sol', 'tetmesh1e2_solution.sol', '-out', 'meshout.mesh', '-hmax', '.7'], returncode=0)

In [13]:
mesh = dotmeshtotetmesh('meshout.mesh')
savesol(solname,mesh,model['strati'])
isovalue =0
meshname='meshout.mesh'
writemultimaterials2('meshout.mmg3d')
subprocess.run(['mmg3d_O3',  meshname,'-ls',str(isovalue), '-sol', solname, '-out', 'meshout.mesh'])

Warning: Meshio doesn't know keyword RequiredVertices. Skipping.

Warning: Meshio doesn't know keyword Tangents. Skipping.

Warning: Meshio doesn't know keyword TangentAtVertices. Skipping.

Warning: Meshio doesn't know keyword Ridges. Skipping.

  -- MMG3D, Release 5.7.0 (Dec. 13, 2022) 
     Copyright (c) Bdx INP/CNRS/Inria/UPMC, 2004-
     Jun 26 2024 16:20:47

  -- INPUT DATA
  %% meshout.mesh OPENED
  %% tetmesh1e2_solution.sol OPENED

  %% meshout.mmg3d OPENED
  -- DATA READING COMPLETED.     0.049s

  -- MMG3DLS: INPUT DATA
  -- INPUT DATA COMPLETED.     0.000s

  -- PHASE 1 : ISOSURFACE DISCRETIZATION

  -- MESH QUALITY  8449
     BEST   0.994479  AVRG.   0.726171  WRST.   0.029839 (808)
  -- PHASE 1 COMPLETED.     0.052s

  -- PHASE 2 : ANALYSIS
  -- PHASE 2 COMPLETED.     0.041s

  -- PHASE 3 : MESH IMPROVEMENT
  *** Manifold implicit surface.
                            3726 splitted,       39 collapsed,    14410 swapped, 5 iter.

  -- GRADATION : 1.300000 (2.300000)
                            1243 splitted,     4215 collapsed,     3348 swapped, 6 iter.
         4013 splitted,     3245 collapsed,     9998 swapped,    24262 moved, 6 iter. 
  *** Manifold implicit surface.
  -- PHASE 3 COMPLETED.     1.441s

  -- MESH

CompletedProcess(args=['mmg3d_O3', 'meshout.mesh', '-ls', '0', '-sol', 'tetmesh1e2_solution.sol', '-out', 'meshout.mesh'], returncode=0)

In [14]:
mesh = dotmeshtotetmesh('meshout.mesh')

Warning: Meshio doesn't know keyword RequiredVertices. Skipping.

Warning: Meshio doesn't know keyword Tangents. Skipping.

Warning: Meshio doesn't know keyword TangentAtVertices. Skipping.

Warning: Meshio doesn't know keyword Ridges. Skipping.

In [15]:
import meshio

mesh_mesh = meshio.read('meshout.mesh')
mesh_mesh.write('meshout.vtk')

Warning: Meshio doesn't know keyword RequiredVertices. Skipping.

Warning: Meshio doesn't know keyword Tangents. Skipping.

Warning: Meshio doesn't know keyword TangentAtVertices. Skipping.

Warning: Meshio doesn't know keyword Ridges. Skipping.

In [16]:
mesh_mesh

<meshio mesh object>
  Number of points: 4785
  Number of cells:
    tetra: 24629
    triangle: 3701
    line: 361
  Point data: medit:ref
  Cell data: medit:ref

In [55]:
mesh_mesh.cells

[<meshio CellBlock, type: tetra, num cells: 24629, tags: []>,
 <meshio CellBlock, type: triangle, num cells: 3701, tags: []>,
 <meshio CellBlock, type: line, num cells: 361, tags: []>]

In [58]:
mesh_mesh.cell_data

{'medit:ref': [array([4, 2, 3, ..., 7, 7, 7]),
  array([ 2,  1,  1, ...,  1,  1, 10]),
  array([10, 10, 10, 10,  0, 10, 10,  0, 10, 10, 10,  0,  0, 10, 10,  0, 10,
          0, 10, 10, 10, 10, 10, 10, 10, 10,  0, 10, 10,  0, 10,  0, 10, 10,
         10, 10, 10, 10, 10, 10,  0, 10,  0,  0, 10,  0, 10, 10, 10,  0,  0,
         10, 10,  0, 10, 10, 10,  0,  0, 10, 10,  0, 10, 10,  0,  0, 10,  0,
          0,  0, 10, 10,  0, 10, 10, 10, 10, 10,  0, 10, 10,  0,  0,  0,  0,
          0,  0, 10, 10, 10, 10, 10, 10, 10,  0, 10, 10, 10, 10, 10, 10,  0,
         10,  0,  0, 10, 10, 10, 10, 10, 10, 10, 10, 10,  0,  0,  0, 10, 10,
          0, 10, 10,  0, 10,  0, 10, 10,  0,  0,  0,  0, 10, 10, 10, 10, 10,
          0, 10, 10, 10,  0, 10,  0, 10, 10, 10, 10,  0, 10, 10, 10,  0, 10,
         10,  0, 10, 10, 10, 10, 10, 10, 10,  0, 10,  0, 10, 10, 10, 10, 10,
         10,  0, 10, 10, 10, 10,  0,  0,  0, 10,  0, 10,  0,  0,  0,  0,  0,
         10, 10, 10, 10, 10,  0, 10, 10,  0, 10,  0, 10,  0,  0,  

In [45]:
print('tetra:')
print(mesh_mesh.cell_data.get('medit:ref')[0].shape)
print(np.unique(mesh_mesh.cell_data.get('medit:ref')[0]))

tetra:
(24629,)
[1 2 3 4 7]


In [46]:
print('triangle:')
print(mesh_mesh.cell_data.get('medit:ref')[1].shape)
print(np.unique(mesh_mesh.cell_data.get('medit:ref')[1]))

triangle:
(3701,)
[ 1  2  4 10]


In [47]:
print('line:')
print(mesh_mesh.cell_data.get('medit:ref')[2].shape)
print(np.unique(mesh_mesh.cell_data.get('medit:ref')[2]))

line:
(361,)
[ 0 10]


In [57]:
mesh_mesh.cells

[<meshio CellBlock, type: tetra, num cells: 24629, tags: []>,
 <meshio CellBlock, type: triangle, num cells: 3701, tags: []>,
 <meshio CellBlock, type: line, num cells: 361, tags: []>]

In [22]:
mesh.nodes*=100

In [23]:
mesh.elements.shape

(24629, 4)

In [24]:
# mesh = TetMesh(tetmesh.nodes,tetmesh.elements)

In [25]:
lower = np.isclose(mesh.nodes[:,2], model.bounding_box.origin[2],atol=1e-3)
upper = np.isclose(mesh.nodes[:,2], model.bounding_box.origin[2] + model.bounding_box.maximum[2],atol=1e-3)
left = np.isclose(mesh.nodes[:,0], model.bounding_box.origin[0],atol=1e-3)
right = np.isclose(mesh.nodes[:,0], model.bounding_box.origin[0] + model.bounding_box.maximum[0],atol=1e-3)
front = np.isclose(mesh.nodes[:,1], model.bounding_box.origin[1],atol=1e-3)
back = np.isclose(mesh.nodes[:,1], model.bounding_box.origin[1] + model.bounding_box.maximum[1],atol=1e-3)
mesh.point_data['Lower'] = lower.astype(int)
mesh.point_data['Upper'] = upper.astype(int)
mesh.point_data['Left'] = left.astype(int)
mesh.point_data['Right'] = right.astype(int)
mesh.point_data['Front'] = front.astype(int)
mesh.point_data['Back'] = back.astype(int)

In [41]:
mesh.point_data['Back'].shape

(4785,)

In [26]:
mesh.save('mesh_after_mmg.exo')

In [59]:
mesh_exo = meshio.read('mesh_after_mmg.exo')

In [64]:
mesh_exo

<meshio mesh object>
  Number of points: 4785
  Number of cells:
    tetra: 24629
  Point data: Lower, Upper, Left, Right, Front, Back

In [73]:
mesh_mesh.point_sets

{}

In [114]:
import datetime

numpy_to_exodus_dtype = {
    "float32": "f4",
    "float64": "f8",
    "int8": "i1",
    "int16": "i2",
    "int32": "i4",
    "int64": "i8",
    "uint8": "u1",
    "uint16": "u2",
    "uint32": "u4",
    "uint64": "u8",
}

exodus_to_meshio_type = {
    "SPHERE": "vertex",
    # curves
    "BEAM": "line",
    "BEAM2": "line",
    "BEAM3": "line3",
    "BAR2": "line",
    # surfaces
    "SHELL": "quad",
    "SHELL4": "quad",
    "SHELL8": "quad8",
    "SHELL9": "quad9",
    "QUAD": "quad",
    "QUAD4": "quad",
    "QUAD5": "quad5",
    "QUAD8": "quad8",
    "QUAD9": "quad9",
    #
    "TRI": "triangle",
    "TRIANGLE": "triangle",
    "TRI3": "triangle",
    "TRI6": "triangle6",
    "TRI7": "triangle7",
    # 'TRISHELL': 'triangle',
    # 'TRISHELL3': 'triangle',
    # 'TRISHELL6': 'triangle6',
    # 'TRISHELL7': 'triangle',
    #
    # volumes
    "HEX": "hexahedron",
    "HEXAHEDRON": "hexahedron",
    "HEX8": "hexahedron",
    "HEX9": "hexahedron9",
    "HEX20": "hexahedron20",
    "HEX27": "hexahedron27",
    #
    "TETRA": "tetra",
    "TETRA4": "tetra4",
    "TET4": "tetra4",
    "TETRA8": "tetra8",
    "TETRA10": "tetra10",
    "TETRA14": "tetra14",
    #
    "PYRAMID": "pyramid",
    "WEDGE": "wedge",
}
meshio_to_exodus_type = {v: k for k, v in exodus_to_meshio_type.items()}



In [208]:
meshio_to_exodus_type['tetra']

'TETRA'

In [134]:
sum(c.data.shape[0] for c in mesh_mesh.cells)

28691

In [180]:
mesh_mesh

<meshio mesh object>
  Number of points: 4785
  Number of cells:
    tetra: 24629
    triangle: 3701
    line: 361
  Point data: medit:ref
  Cell data: medit:ref

In [157]:
element_labels = mesh_mesh.cell_data.get('medit:ref')[0]
labels_arr = np.unique(element_labels)
print(labels_arr)

[1 2 3 4 7]


In [158]:
element_labels

array([4, 2, 3, ..., 7, 7, 7])

In [172]:
np.unique(mesh_mesh.cells[0].data)

array([   0,    1,    2, ..., 4782, 4783, 4784])

In [173]:
mesh_mesh.points

array([[0.00000000e+00, 1.83670992e-40, 0.00000000e+00],
       [1.00000000e+01, 1.83670992e-40, 0.00000000e+00],
       [1.00000000e+01, 1.00000000e+01, 0.00000000e+00],
       ...,
       [9.61159065e+00, 3.50132875e+00, 7.77172206e-02],
       [4.88537713e+00, 3.82409942e+00, 7.06161947e-01],
       [4.55325423e+00, 5.78395575e+00, 1.00533470e+00]])

In [178]:
for label in labels_arr:
    label_elements = mesh_mesh.cells[0].data[np.where(element_labels == label)]
    # print(label_elements)
    # print(np.unique(label_elements))
    label_points = mesh_mesh.points[np.unique(label_elements)]
    # print(label_points)

In [182]:
label_elements.dtype.name

'int64'

In [187]:
label_points.T

array([[0.00000000e+00, 0.00000000e+00, 6.05691557e+00, ...,
        5.48351920e+00, 5.58662579e+00, 3.61614427e+00],
       [1.83670992e-40, 1.00000000e+01, 3.21370049e+00, ...,
        7.76620721e+00, 7.91199474e-02, 5.19246108e+00],
       [2.00000000e+00, 2.00000000e+00, 2.00000000e+00, ...,
        1.23381121e+00, 1.01880308e+00, 1.39751723e+00]])

In [188]:
label_points.shape[0]

1818

In [197]:
mesh_mesh.cells[0].data.shape

(24629, 4)

In [217]:
for label in [labels_arr[0]]:
    print(label)

1


In [239]:
def convert_mesh2exo(filename, mesh):
    import netCDF4

    with netCDF4.Dataset(filename, "w") as rootgrp:
        # set global data
        now = datetime.datetime.now().isoformat()
        rootgrp.title = f"Created by .mesh file, {now}"
        rootgrp.version = np.float32(5.1)
        rootgrp.api_version = np.float32(5.1)
        rootgrp.floating_point_word_size = 8
        rootgrp.createDimension("len_string", 33)
        rootgrp.createDimension("len_line", 81)
        

        # finding unique labels in tetra
        element_labels = mesh.cell_data.get('medit:ref')[0]
        labels_arr = np.unique(element_labels)

        rootgrp.createDimension("num_nodes", len(mesh.points))
        
        # separate each label elements and points into blocks
        for i, label in enumerate(labels_arr):
            label_elements = mesh_mesh.cells[0].data[np.where(element_labels == label)]
            label_points = mesh_mesh.points[np.unique(label_elements)]
            
            rootgrp.createDimension(f"num_el_blk{i}", label_elements.shape[0])
            rootgrp.createDimension(f"num_nodes{i}", label_points.shape[0])
            rootgrp.createDimension(f"num_dim{i}", label_points.shape[1])
            rootgrp.createDimension(f"time_step{i}", None)
    
    
            # dummy time step
            data = rootgrp.createVariable(f"time_whole{i}", "f4", (f"time_step{i}",))
            data[:] = 0.0
    
            # points
            coor_names = rootgrp.createVariable(f"coor_names{i}", "S1", (f"num_dim{i}", "len_string"))
            coor_names.set_auto_mask(False)
            coor_names[0, 0] = b"X"
            coor_names[1, 0] = b"Y"
            if mesh.points.shape[1] == 3:
                coor_names[2, 0] = b"Z"
            data = rootgrp.createVariable(f"coord{i}", numpy_to_exodus_dtype[label_points.dtype.name], (f"num_dim{i}", f"num_nodes{i}"),)
            data[:] = label_points.T

            # cells
            # ParaView needs eb_prop1 -- some ID. The values don't seem to matter as
            # long as they are different for the for different blocks.
            data = rootgrp.createVariable(f"eb_prop{i}", "i4", f"num_el_blk{i}")
            data[:] = i
            for k, cell_block in enumerate([label_elements]):
                dim1 = f"num_el_in_blk{i}{k + 1}"
                dim2 = f"num_nod_per_el{i}{k + 1}"
                rootgrp.createDimension(dim1, cell_block.shape[0])
                rootgrp.createDimension(dim2, cell_block.shape[1])
                dtype = numpy_to_exodus_dtype[cell_block.dtype.name]
                data = rootgrp.createVariable(f"connect{i}{k + 1}", dtype, (dim1, dim2))
                data.elem_type = meshio_to_exodus_type['tetra']
                # Exodus is 1-based
                data[:] = cell_block + 1

import os 
os.remove('ex_mesh2exo.exo')
convert_mesh2exo('ex_mesh2exo.exo', mesh_mesh)

In [214]:
def write(filename, mesh):
    import netCDF4

    with netCDF4.Dataset(filename, "w") as rootgrp:
        # set global data
        now = datetime.datetime.now().isoformat()
        rootgrp.title = f"Created by meshio v5.3.5, {now}"
        rootgrp.version = np.float32(5.1)
        rootgrp.api_version = np.float32(5.1)
        rootgrp.floating_point_word_size = 8

        # set dimensions
        total_num_elems = sum(c.data.shape[0] for c in [mesh.cells[0]])
        rootgrp.createDimension("num_nodes", len(mesh.points))
        rootgrp.createDimension("num_dim", mesh.points.shape[1])
        rootgrp.createDimension("num_elem", total_num_elems)
        rootgrp.createDimension("num_el_blk", len([mesh.cells[0]]))
        rootgrp.createDimension("num_node_sets", len(mesh.point_sets))
        rootgrp.createDimension("len_string", 33)
        rootgrp.createDimension("len_line", 81)
        rootgrp.createDimension("four", 4)
        rootgrp.createDimension("time_step", None)

        # dummy time step
        data = rootgrp.createVariable("time_whole", "f4", ("time_step",))
        data[:] = 0.0

        # points
        coor_names = rootgrp.createVariable(
            "coor_names", "S1", ("num_dim", "len_string")
        )
        coor_names.set_auto_mask(False)
        coor_names[0, 0] = b"X"
        coor_names[1, 0] = b"Y"
        if mesh.points.shape[1] == 3:
            coor_names[2, 0] = b"Z"
        data = rootgrp.createVariable(
            "coord",
            numpy_to_exodus_dtype[mesh.points.dtype.name],
            ("num_dim", "num_nodes"),
        )
        data[:] = mesh.points.T

        # cells
        # ParaView needs eb_prop1 -- some ID. The values don't seem to matter as
        # long as they are different for the for different blocks.
        data = rootgrp.createVariable("eb_prop1", "i4", "num_el_blk")
        for k in range(len([mesh.cells[0]])):
            data[k] = k
        for k, cell_block in enumerate([mesh.cells[0]]):
            dim1 = f"num_el_in_blk{k + 1}"
            dim2 = f"num_nod_per_el{k + 1}"
            rootgrp.createDimension(dim1, cell_block.data.shape[0])
            rootgrp.createDimension(dim2, cell_block.data.shape[1])
            dtype = numpy_to_exodus_dtype[cell_block.data.dtype.name]
            data = rootgrp.createVariable(f"connect{k + 1}", dtype, (dim1, dim2))
            data.elem_type = meshio_to_exodus_type[cell_block.type]
            # Exodus is 1-based
            data[:] = cell_block.data + 1

        # # point data
        # # The variable `name_nod_var` holds the names and indices of the node variables, the
        # # variables `vals_nod_var{1,2,...}` hold the actual data.
        # num_nod_var = len(mesh.point_data)
        # if num_nod_var > 0:
        #     rootgrp.createDimension("num_nod_var", num_nod_var)
        #     # set names
        #     point_data_names = rootgrp.createVariable(
        #         "name_nod_var", "S1", ("num_nod_var", "len_string")
        #     )
        #     point_data_names.set_auto_mask(False)
        #     for k, name in enumerate(mesh.point_data.keys()):
        #         for i, letter in enumerate(name):
        #             point_data_names[k, i] = letter.encode()

        #     # Set data. ParaView might have some problems here, see
        #     # <https://gitlab.kitware.com/paraview/paraview/-/issues/18403>.
        #     for k, (name, data) in enumerate(mesh.point_data.items()):
        #         for i, s in enumerate(data.shape):
        #             rootgrp.createDimension(f"dim_nod_var{k}{i}", s)
        #         dims = ["time_step"] + [
        #             f"dim_nod_var{k}{i}" for i in range(len(data.shape))
        #         ]
        #         node_data = rootgrp.createVariable(
        #             f"vals_nod_var{k + 1}",
        #             numpy_to_exodus_dtype[data.dtype.name],
        #             tuple(dims),
        #             fill_value=False,
        #         )
        #         node_data[0] = data

        # # node sets
        # num_point_sets = len(mesh.point_sets)
        # if num_point_sets > 0:
        #     data = rootgrp.createVariable("ns_prop1", "i4", "num_node_sets")
        #     data_names = rootgrp.createVariable(
        #         "ns_names", "S1", ("num_node_sets", "len_string")
        #     )
        #     for k, name in enumerate(mesh.point_sets.keys()):
        #         data[k] = k
        #         for i, letter in enumerate(name):
        #             data_names[k, i] = letter.encode()
        #     for k, (key, values) in enumerate(mesh.point_sets.items()):
        #         dim1 = f"num_nod_ns{k + 1}"
        #         rootgrp.createDimension(dim1, values.shape[0])
        #         dtype = numpy_to_exodus_dtype[values.dtype.name]
        #         data = rootgrp.createVariable(f"node_ns{k + 1}", dtype, (dim1,))
        #         # Exodus is 1-based
        #         data[:] = values + 1

import os 
os.remove('aaaaa_test_tg.exo')
write('aaaaa_test_tg.exo', mesh_mesh)

In [242]:
import netCDF4
import numpy as np

# Define the Exodus II file name
exodus_filename = 'mesh_with_two_blocks.exo'

# Create a new NetCDF file
rootgrp = netCDF4.Dataset(exodus_filename, mode='w', format='NETCDF4')

# Define dimensions
num_dim = 2
num_nodes = 6
num_elem_blk = 2
num_nodes_per_elem = 4
num_elem1 = 1
num_elem2 = 1

rootgrp.createDimension('num_dim', num_dim)
rootgrp.createDimension('num_nodes', num_nodes)
rootgrp.createDimension('num_elem_blk', num_elem_blk)
rootgrp.createDimension('num_elem_in_blk1', num_elem1)
rootgrp.createDimension('num_elem_in_blk2', num_elem2)
rootgrp.createDimension('num_nod_per_el1', num_nodes_per_elem)
rootgrp.createDimension('num_nod_per_el2', num_nodes_per_elem)
rootgrp.createDimension('len_string', 33)
rootgrp.createDimension('len_line', 81)

# Define variables for coordinates
coords = rootgrp.createVariable('coord', 'f4', ('num_dim', 'num_nodes'))
coord_names = rootgrp.createVariable('coor_names', 'S1', ('num_dim', 'len_string'))

# Define variables for element connectivity
connect1 = rootgrp.createVariable('connect1', 'i4', ('num_elem_in_blk1', 'num_nod_per_el1'))
connect2 = rootgrp.createVariable('connect2', 'i4', ('num_elem_in_blk2', 'num_nod_per_el2'))

# Define variables for element block properties
eb_prop1 = rootgrp.createVariable('eb_prop1', 'i4', 'num_elem_blk')

# Define element variable names
elem_var_names = rootgrp.createVariable('name_elem_var', 'S1', ('num_elem_blk', 'len_string'))
elem_var1 = rootgrp.createVariable('vals_elem_var1', 'f4', ('num_elem_in_blk1',))
elem_var2 = rootgrp.createVariable('vals_elem_var2', 'f4', ('num_elem_in_blk2',))

# Assign coordinate data
coords[0, :] = [0.0, 1.0, 1.0, 0.0, 2.0, 2.0]  # x-coordinates
coords[1, :] = [0.0, 0.0, 1.0, 1.0, 0.0, 1.0]  # y-coordinates

# Assign coordinate names
coord_names[0, :4] = netCDF4.stringtochar(np.array(['x   '], 'S1'))
coord_names[1, :4] = netCDF4.stringtochar(np.array(['y   '], 'S1'))

# Assign connectivity data
connect1[0, :] = [1, 2, 3, 4]  # First quadrilateral element
connect2[0, :] = [2, 5, 6, 3]  # Second quadrilateral element

# Assign element block properties
eb_prop1[:] = [1, 2]  # Unique IDs for element blocks

# Assign element variable names
elem_var_names[0, :11] = netCDF4.stringtochar(np.array(['temperature'], 'S1'))
elem_var_names[1, :11] = netCDF4.stringtochar(np.array(['temperature'], 'S1'))

# Assign element variable values (example: temperature values for elements)
elem_var1[:] = [100.0]  # Temperature for elements in block 1
elem_var2[:] = [200.0]  # Temperature for elements in block 2

# Add global attributes
rootgrp.title = 'Mesh with Two Blocks Example'
rootgrp.api_version = np.float32(5.10)
rootgrp.floating_point_word_size = np.int32(8)
rootgrp.file_size = 1

# Ensure unique block names
elem_block_names = rootgrp.createVariable('eb_names', 'S1', ('num_elem_blk', 'len_string'))
unique_block_name1 = 'block_1'
unique_block_name2 = 'block_2'
elem_block_names[0, :len(unique_block_name1)] = netCDF4.stringtochar(np.array(list(unique_block_name1), 'S1'))
elem_block_names[1, :len(unique_block_name2)] = netCDF4.stringtochar(np.array(list(unique_block_name2), 'S1'))

# Close the file
rootgrp.close()


In [244]:
import netCDF4
import numpy as np

# Define the Exodus II file name
exodus_filename = 'mesh_with_two_blocks2.exo'

# Create a new NetCDF file
rootgrp = netCDF4.Dataset(exodus_filename, mode='w', format='NETCDF4')

# Define dimensions
num_dim = 2
num_nodes = 6
num_elem_blk = 2
num_nodes_per_elem = 4
num_elem1 = 1
num_elem2 = 1

rootgrp.createDimension('num_dim', num_dim)
rootgrp.createDimension('num_nodes', num_nodes)
rootgrp.createDimension('num_elem_blk', num_elem_blk)
rootgrp.createDimension('num_el_in_blk1', num_elem1)
rootgrp.createDimension('num_el_in_blk2', num_elem2)
rootgrp.createDimension('num_nod_per_el1', num_nodes_per_elem)
rootgrp.createDimension('num_nod_per_el2', num_nodes_per_elem)
rootgrp.createDimension('len_string', 33)
rootgrp.createDimension('len_line', 81)
rootgrp.createDimension('four', 4)

# Define variables for coordinates
coords = rootgrp.createVariable('coord', 'f4', ('num_dim', 'num_nodes'))
coord_names = rootgrp.createVariable('coor_names', 'S1', ('num_dim', 'len_string'))

# Define variables for element connectivity
connect1 = rootgrp.createVariable('connect1', 'i4', ('num_el_in_blk1', 'num_nod_per_el1'))
connect2 = rootgrp.createVariable('connect2', 'i4', ('num_el_in_blk2', 'num_nod_per_el2'))

# Define variables for element block properties
eb_prop1 = rootgrp.createVariable('eb_prop1', 'i4', 'num_elem_blk')

# Define element variable names
elem_var_names = rootgrp.createVariable('name_elem_var', 'S1', ('num_elem_blk', 'len_string'))
elem_var1 = rootgrp.createVariable('vals_elem_var1', 'f4', ('num_el_in_blk1',))
elem_var2 = rootgrp.createVariable('vals_elem_var2', 'f4', ('num_el_in_blk2',))

# Assign coordinate data
coords[0, :] = [0.0, 1.0, 1.0, 0.0, 2.0, 2.0]  # x-coordinates
coords[1, :] = [0.0, 0.0, 1.0, 1.0, 0.0, 1.0]  # y-coordinates

# Assign coordinate names
coord_names[0, :4] = netCDF4.stringtochar(np.array(list('x'), 'S1'))
coord_names[1, :4] = netCDF4.stringtochar(np.array(list('y'), 'S1'))

# Assign connectivity data
connect1[0, :] = [1, 2, 3, 4]  # First quadrilateral element
connect2[0, :] = [2, 5, 6, 3]  # Second quadrilateral element

# Assign element block properties
eb_prop1[:] = [1, 2]  # Unique IDs for element blocks

# Assign element variable names
elem_var_names[0, :11] = netCDF4.stringtochar(np.array(list('temperature'), 'S1'))
elem_var_names[1, :11] = netCDF4.stringtochar(np.array(list('temperature'), 'S1'))

# Assign element variable values (example: temperature values for elements)
elem_var1[:] = [100.0]  # Temperature for elements in block 1
elem_var2[:] = [200.0]  # Temperature for elements in block 2

# Add global attributes
rootgrp.title = 'Mesh with Two Blocks Example'
rootgrp.api_version = np.float32(5.10)
rootgrp.version = np.float32(5.10)
rootgrp.floating_point_word_size = np.int32(8)
rootgrp.file_size = 1

# Ensure unique block names
elem_block_names = rootgrp.createVariable('eb_names', 'S1', ('num_elem_blk', 'len_string'))
unique_block_name1 = 'block_1'
unique_block_name2 = 'block_2'
elem_block_names[0, :len(unique_block_name1)] = netCDF4.stringtochar(np.array(list(unique_block_name1), 'S1'))
elem_block_names[1, :len(unique_block_name2)] = netCDF4.stringtochar(np.array(list(unique_block_name2), 'S1'))

# Close the file
rootgrp.close()
